In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


---

# 모듈로딩

In [5]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets.boston_housing import load_data
import numpy as np

# [1] 데이터로딩

In [7]:
# 학습용, 테스트용 = 8:2
(x_train, y_train),(x_test,y_test) = load_data()


65536/57026 [==================================] - 0s 0us/step


In [8]:
x_train.shape,x_test.shape

((404, 13), (102, 13))

In [9]:
x_train[:1],y_train[0]

(array([[  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
          91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
          18.72   ]]), 15.2)

# [2] NN Model 실습
---
- (1) 모델 구상
- (2) 모델 생성
- (3) 학습 
- (4) 평가
- (5) 테스트

##### [2-1] 모델 구상
- 입력 데이터 => 13개 features
- 출력 데이터 => 1개 집값
- 학습 방법   => 회귀 분류

Dense Layer : node=1,  
af = linear,  
input_shape = (13,)

In [11]:
# NN Model의 Layer를 층층이 담아서 관리하는 객체
model = Sequential(name = 'Boston')

In [13]:
# Layer 추가
model.add(Dense(1,activation='linear',input_shape=(13,)))

In [14]:
model.summary()

Model: "Boston"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 14        
                                                                 
Total params: 14
Trainable params: 14
Non-trainable params: 0
_________________________________________________________________


#### [2-2] 모델 생성
---
#### Sequential 클래스의 compile 메서드
    - 모델이 동작하기 위해서 필요한 설정
    1. 오차 계산 함수 설정 = loss
                     
    2. W, B 최적화 방식 설정 => optimizer
  
    3. 모델의 성능 측정 항목 설정 => metrics
      * 파라미터 값들
        - str 타입으로 입력 가능
        - 객체 타입으로 입력 가능

#### 오차/로스계한함수 loss param
    - 회귀 -> mae, mse, rmse
    - 분류 -> crossentropy 
        - 이진분류: binary_crossentropy
        - 다중분류: categorical_crossentropy(target 원핫인코딩 시), sparse_categorical_crossentropy  

#### 최적화
    - 기본: rmsprop
    - adam, momonent,... 최적화 클래스 이름 문자열 입력  


In [29]:
model.compile(loss = 'mse', optimizer = 'adam',metrics = 'accuracy')

#### [2-3]학습
---
    - fit() 메서드
      * 파라미터
      * 반환값 => History 객체

In [30]:
result = model.fit(x_train,y_train, epochs=3)

Epoch 1/3
13/13 [==============================] - 0s 2ms/step - loss: 7344.4497 - accuracy: 0.0000e+00
Epoch 2/3
13/13 [==============================] - 0s 2ms/step - loss: 6627.1338 - accuracy: 0.0000e+00
Epoch 3/3
13/13 [==============================] - 0s 2ms/step - loss: 6117.4858 - accuracy: 0.0000e+00


In [31]:
type(result), result.history, result.params

(keras.callbacks.History,
 {'loss': [7344.44970703125, 6627.1337890625, 6117.48583984375],
  'accuracy': [0.0, 0.0, 0.0]},
 {'verbose': 1, 'epochs': 3, 'steps': 13})

In [32]:
result.history.keys()

dict_keys(['loss', 'accuracy'])

In [33]:
# keys로 값 확인 -> 객체.history[keyname]
result.history['loss']

[7344.44970703125, 6627.1337890625, 6117.48583984375]

In [34]:
result.history['accuracy']

[0.0, 0.0, 0.0]